In [46]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
!pip install geopy[aiohttp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import requests
import time
import json

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import concurrent.futures

API_NOMINATIM =  'https://nominatim.openstreetmap.org/'

In [2]:
dataset = pd.read_csv('/content/Building_Permits_v5 .csv', low_memory=False)

In [3]:
idx = np.where((dataset["Proposed Use"]=="1 family dwelling") | (dataset["Proposed Use"]=="2 family dwelling") | (dataset["Proposed Use"]=="apartments"))
dataset = dataset.loc[idx]

In [4]:
dataset.shape

(7561, 43)

# Tallest buildings in SF = 61 floors
[source wikipedia](https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_San_Francisco)

In [5]:
outliers_floors = dataset["Number of Proposed Stories"] < 61
dataset = dataset[outliers_floors]
dataset.shape

(7561, 43)

# Delete Estimated cost Outliers
cost <= 5 000 $

In [6]:
mask_cost = dataset["Est_Cost_Infl"] >= 5000
dataset=dataset[mask_cost]
dataset.shape

(7561, 43)

In [7]:
dataset['Zipcode'] = dataset['Zipcode'].replace(np.nan, 0.0).astype(str)

In [8]:
dataset['Zipcode'] = dataset['Zipcode'].apply(lambda x: x[:-2])

In [9]:
dataset['Zipcode'] = dataset['Zipcode'].replace('0','')

In [10]:
dataset['Zipcode'].head()

0    94114
1    94122
2    94133
3    94118
4    94107
Name: Zipcode, dtype: object

In [11]:
dataset['full_address'] = dataset['address'] + ', San Francisco, CA ' + dataset['Zipcode'].astype(str)

In [12]:
columns = ['Year','full_address','Street Number','Street Name','Street Suffix','Zipcode','lat','lon', 'Neighborhoods - Analysis Boundaries','Proposed Units','Proposed Use','Number of Proposed Stories','Est_Cost_Infl']
dataset[columns].head()

,Year,full_address,Street Number,Street Name,Street Suffix,Zipcode,lat,lon,Neighborhoods - Analysis Boundaries,Proposed Units,Proposed Use,Number of Proposed Stories,Est_Cost_Infl
0,1991,"3686 17th St, San Francisco, CA 94114",3686,17th,St,94114,-122.427980,37.763166,Castro/Upper Market,3.0,apartments,4.0,4.973752e+05
1,1987,"1254 18th Av, San Francisco, CA 94122",1254,18th,Av,94122,-122.475786,37.764472,Inner Sunset,4.0,apartments,3.0,9.398079e+05
2,1977,"1059 Union St, San Francisco, CA 94133",1059,Union,St,94133,-122.416534,37.799258,Russian Hill,12.0,apartments,4.0,7.476268e+06
3,1979,"3900 California St, San Francisco, CA 94118",3900,California,St,94118,-122.459525,37.785859,Inner Richmond,8.0,apartments,3.0,2.948572e+06
4,1980,"1084 De Haro St, San Francisco, CA 94107",1084,De Haro,St,94107,-122.401077,37.756079,Potrero Hill,2.0,2 family dwelling,3.0,6.922231e+05


In [18]:
splited_address = np.split(dataset['full_address'].values, [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000], axis=0)

In [20]:
len(splited_address)

15

In [35]:
# TEST
def get_geojson_list(addresses):
    """
        Get the features from Api Geo
    """
    geojson_list = []

    for address in addresses:
      print(f'geojson_list taille = {len(geojson_list)}')
      print(f'{address}')
      try:
        response = requests.get(API_NOMINATIM + 'search', params={'q':address, 'format': 'geojson', 'polygon_geojson':'1'}).json()['features'][0]
        if "polygon" in response['geometry']['type'].lower() : 
          print(response['geometry']['type'].lower())
          geojson_list.append(json.dumps(response['geometry']))
        else:
          print("pas de polygon")
          geojson_list.append(np.nan)
      except:
        print("echec")
        geojson_list.append(np.nan)
      print('\n')
    return geojson_list


In [37]:
geolocator = Nominatim(user_agent="Building Permits San Francisco")
start_time = time.time()
with concurrent.futures.ThreadPoolExecutor() as e:
    locations = list(e.map(get_geojson_list, splited_address))
    
print("--- %s seconds ---" % (time.time() - start_time))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.


geojson_list taille = 314
157 Apollo St, San Francisco, CA 94124
pas de polygon


geojson_list taille = 314
231 Scott St, San Francisco, CA 94117
pas de polygon


geojson_list taille = 311
150 Hawthorne St, San Francisco, CA 94107
pas de polygon


geojson_list taille = 311
1545 Pine St, San Francisco, CA 94109
polygon


geojson_list taille = 314
70 Waterville St, San Francisco, CA 94124
pas de polygon


geojson_list taille = 315
3645 23rd St, San Francisco, CA 94110
pas de polygon


geojson_list taille = 315
916 Corbett Av, San Francisco, CA 94131
pas de polygon


geojson_list taille = 312
447 Monterey Blvd, San Francisco, CA 94127
pas de polygon


geojson_list taille = 312
580 Hayes St, San Francisco, CA 94102
polygon


geojson_list taille = 315
1454 Quesada Av, San Francisco, CA 94124
pas de polygon


geojson_list taille = 316
3647 23rd St, San Francisco, CA 94110
polygon


geojson_list taille = 316
1088 

In [38]:
len(locations)

15

In [39]:
geojson_list = [feature for result in locations for feature in result]

In [40]:
len(geojson_list)

7561

In [41]:
dataset["geometry"] = geojson_list

In [46]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7561 entries, 0 to 7560
Data columns (total 44 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Permit Number                           7561 non-null   object 
 1   Permit Type                             7561 non-null   int64  
 2   Permit Type Definition                  7561 non-null   object 
 3   Permit Creation Date                    7561 non-null   object 
 4   Block                                   7561 non-null   object 
 5   Lot                                     7561 non-null   object 
 6   Street Number                           7561 non-null   int64  
 7   Street Name                             7561 non-null   object 
 8   Street Suffix                           7561 non-null   object 
 9   Description                             6418 non-null   object 
 10  Current Status                          7561 non-null   obje

In [47]:
dataset.to_csv('Building_Permits_v6.csv',index=False)